In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
!pip install -U ipykernel #updating my ikernel

     |████████████████████████████████| 122kB 7.5MB/s 
     |████████████████████████████████| 788kB 34.6MB/s 
     |████████████████████████████████| 368kB 50.2MB/s 
ERROR: jupyter-console 5.2.0 has requirement prompt-toolkit<2.0.0,>=1.0.0, but you'll have prompt-toolkit 3.0.19 which is incompatible.
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 6.0.2 which is incompatible.
ERROR: google-colab 1.0.0 has requirement ipython~=5.5.0, but you'll have ipython 7.25.0 which is incompatible.
  Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      Successfully uninstalled prompt-toolkit-1.0.18
  Found existing installation: ipython 5.5.0
    Uninstalling ipython-5.5.0:
      Successfully uninstalled ipython-5.5.0
  Found existing installation: importlib-metadata 4.6.0
    Uninstalling importlib-metadata-4.6.0:
      Successfully uninstalled importlib-metadata-4.6.0
  Found existing installation: ipykernel 4.10.1


In [2]:
!pip install -q streamlit #this is where well deploy

     |████████████████████████████████| 7.8MB 8.2MB/s 
     |████████████████████████████████| 81kB 11.7MB/s 
     |████████████████████████████████| 112kB 51.9MB/s 
     |████████████████████████████████| 4.2MB 39.7MB/s 
     |████████████████████████████████| 174kB 50.2MB/s 
     |████████████████████████████████| 122kB 53.7MB/s 
     |████████████████████████████████| 71kB 10.6MB/s 
ERROR: ipykernel 6.0.2 has requirement ipython<8.0,>=7.23.1, but you'll have ipython 5.5.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 6.0.2 which is incompatible.


In [3]:
!pip install pyngrok==4.1.1

  Created wheel for pyngrok: filename=pyngrok-4.1.1-cp37-none-any.whl size=15985 sha256=bbef556be05495cc8197e4784904d60b1a8c3121d8053394a217a246b0a165d2
  Stored in directory: /root/.cache/pip/wheels/97/71/0d/1695f7c8815c0beb3b5d9b35d6eec9243c87e6070fbe3977fa
Successfully built pyngrok


In [4]:
!ngrok authtoken <Enter your token here>

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
%%writefile score.py 

import tensorflow as tf
import numpy as np
import streamlit as st
from PIL import Image
import requests
from io import BytesIO

st.set_option('deprecation.showfileUploaderEncoding', False)
st.title("Bean Image Classifier")
st.text("Provide URL of bean Image for image classification")

@st.cache(allow_output_mutation=True) #cache the model, so you dont have to load the model again and again
def load_model():
  model = tf.keras.models.load_model('/content/drive/My Drive/Colab Notebooks/models/image/models') 
  return model

with st.spinner('Loading Model Into Memory....'):
  model = load_model()

classes=['angular_leaf_spot','bean_rust','healthy']

def scale(image):
  image = tf.cast(image, tf.float32)
  image /= 255.0

  return tf.image.resize(image,[224,224])

def decode_img(image):
  img = tf.image.decode_jpeg(image, channels=3)
  img = scale(img)
  return np.expand_dims(img, axis=0)

path = st.text_input('Enter Image URL to Classify.. ','http://barmac.com.au/wp-content/uploads/sites/3/2016/01/Angular-Leaf-Spot-Beans1.jpg')
if path is not None:
    content = requests.get(path).content

    st.write("Predicted Class :")
    with st.spinner('classifying.....'):
      label =np.argmax(model.predict(decode_img(content)),axis=1)
      st.write(classes[label[0]])    
    st.write("")
    image = Image.open(BytesIO(content))
    st.image(image, caption='Classifying Bean Image', use_column_width=True)

Writing score.py


In [7]:
!nohup streamlit run score.py & 

nohup: appending output to 'nohup.out'


In [12]:
from pyngrok import ngrok

url = ngrok.connect(port=8501) #8501 is the default port for streamlit
url

'http://3b634549d20e.ngrok.io'